In [1]:
import os
import json
from utils import json_load, json_dump, mkdir

### 1. load exam data first

In [2]:
K = 10
SOURCE_DIR = os.path.join('data', 'source')
exam_dataset_file_path = os.path.join(SOURCE_DIR, 'exam_dataset.json')
exam_data = json_load(exam_dataset_file_path)
exam_data = exam_data['examples'][:K]
exam_data

load data from: data/source/exam_dataset.json


[{'query': '題目: 常見針灸配穴法中,所指的「四關穴」,為下列何穴位之組合?\n選項:\n A: 上星、日月\n B: 合谷、太衝\n C: 內關、外關\n D: 上關、下關\n',
  'query_by': {'model_name': 're', 'type': 'ai'},
  'reference_contexts': None,
  'reference_answer': 'B',
  'reference_answer_by': {'model_name': 're', 'type': 'ai'}},
 {'query': '題目: 依《靈樞.經脈》記載,「其直者,從巔入絡腦,還出別下項,循肩膊內,挾脊抵腰中」,指下列 何經的循行內容?\n選項:\n A: 膀胱經\n B: 膽經\n C: 胃經\n D: 肝經\n',
  'query_by': {'model_name': 're', 'type': 'ai'},
  'reference_contexts': None,
  'reference_answer': 'A',
  'reference_answer_by': {'model_name': 're', 'type': 'ai'}},
 {'query': '題目: 依《靈樞.經脈》所記載:「是主筋所生病者,痔、瘧、狂、癲疾、頭顖、項痛,目黃、淚出,鼽 衄,項、背、腰、尻、膕、腨、腳皆痛,小趾不用」,指何經的病證內容?\n選項:\n A: 膀胱經\n B: 膽經\n C: 三焦經\n D: 脾經\n',
  'query_by': {'model_name': 're', 'type': 'ai'},
  'reference_contexts': None,
  'reference_answer': 'A',
  'reference_answer_by': {'model_name': 're', 'type': 'ai'}},
 {'query': '題目: 有關腧穴之國際譯名,外丘之編號為何?\n選項:\n A: GB33\n B: GB34\n C: GB35\n D: GB36\n',
  'query_by': {'model_name': 're', 'type': 'ai'},
  'reference_

### 2. answer directly

In [3]:
import os
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

from llama_index.llms.openai import OpenAI
from llama_index.llms.ollama import Ollama

llm = OpenAI(model="gpt-5-mini", temperature=0, is_streaming=False, response_format={"type": "json_object"})
#llm = Ollama(model="gemma3:12b", temperature=0.0, request_timeout=120.0, context_window=8000, json_mode=True)

In [4]:
from llama_index.core.prompts import PromptTemplate

ANSWER_PROMPT = PromptTemplate(
    template="""
你是一個中醫考題專家，請根據下面的題目回答單選題。
要求：
1. 輸出 JSON 格式
2. key: "ans" 只回答單選答案 (A/B/C/D)
3. key: "feedback" 簡短說明為什麼選這個答案
4. 不要輸出其他文字

#{query}

請直接輸出 JSON：
"""
)

In [5]:
ANSWER_PROMPT

PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['query'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='\n你是一個中醫考題專家，請根據下面的題目回答單選題。\n要求：\n1. 輸出 JSON 格式\n2. key: "ans" 只回答單選答案 (A/B/C/D)\n3. key: "feedback" 簡短說明為什麼選這個答案\n4. 不要輸出其他文字\n\n#{query}\n\n請直接輸出 JSON：\n')

In [6]:
rvs = []
for idx, exam in enumerate(exam_data):
    print(f'idx: {idx}')
    rv = {}
    query = exam['query']
    prompt = ANSWER_PROMPT.format(query=query)
    # print(prompt)
    response = llm.complete(prompt)
    rv['query'] = query
    rv['reference_answer'] = exam['reference_answer']
    rv['response'] = json.loads(response.text)
    rvs.append(rv)

idx: 0
idx: 1
idx: 2
idx: 3
idx: 4
idx: 5
idx: 6
idx: 7
idx: 8
idx: 9


In [7]:
DEST_DIR = os.path.join('data', 'deliverables')
mkdir(DEST_DIR)
save_file_path = os.path.join(DEST_DIR, 'mini_response_directly.json')
json_dump(save_file_path, rvs)

Directory 'data/deliverables' already exists.
dump result to: data/deliverables/mini_response_directly.json


# check the answer

In [8]:
correct = 0
incorrect = 0
for rv in rvs:
    reference_answer = rv['reference_answer']
    pred_ans = rv['response']['ans']
    if reference_answer == pred_ans:
        correct+=1
    else:
        incorrect+=1

print(f"{correct}/{correct + incorrect}")

7/10


In [9]:
rvs

[{'query': '題目: 常見針灸配穴法中,所指的「四關穴」,為下列何穴位之組合?\n選項:\n A: 上星、日月\n B: 合谷、太衝\n C: 內關、外關\n D: 上關、下關\n',
  'reference_answer': 'B',
  'response': {'ans': 'B',
   'feedback': '「四關」即四門/四關法，為雙側合谷（LI4）與太衝（LR3）共四穴，用以疏通氣機、止痛與調節全身氣血。'}},
 {'query': '題目: 依《靈樞.經脈》記載,「其直者,從巔入絡腦,還出別下項,循肩膊內,挾脊抵腰中」,指下列 何經的循行內容?\n選項:\n A: 膀胱經\n B: 膽經\n C: 胃經\n D: 肝經\n',
  'reference_answer': 'A',
  'response': {'ans': 'A',
   'feedback': '記述為從巔入絡腦、出項循肩挾脊至腰者，乃膀胱經入腦並沿脊旁下行的經脈特徵。'}},
 {'query': '題目: 依《靈樞.經脈》所記載:「是主筋所生病者,痔、瘧、狂、癲疾、頭顖、項痛,目黃、淚出,鼽 衄,項、背、腰、尻、膕、腨、腳皆痛,小趾不用」,指何經的病證內容?\n選項:\n A: 膀胱經\n B: 膽經\n C: 三焦經\n D: 脾經\n',
  'reference_answer': 'A',
  'response': {'ans': 'B',
   'feedback': '描述有目黃（黃疸）、頭項側痛及沿身側、下肢放射性疼痛等，符合膽經的循行與病證。'}},
 {'query': '題目: 有關腧穴之國際譯名,外丘之編號為何?\n選項:\n A: GB33\n B: GB34\n C: GB35\n D: GB36\n',
  'reference_answer': 'D',
  'response': {'ans': 'D', 'feedback': '外丘屬膽經下肢穴，國際腧穴編號為GB36，因此答案為D。'}},
 {'query': '題目: 下列何者為陰維脈之郄穴?\n選項:\n A: 交信\n B: 築賓\n C: 府舍\n D: 腹哀\n',
  'reference_answer': 'B',
  'response': {'ans':